In [6]:
import pandas as pd

data = pd.read_csv('./data/Congressional Voting Records.csv')

data_y = data['Class']

data = data.drop('Class', axis=1)

# 把data class補到最後一行

data['class'] = data_y

data.to_csv('./data/Congressional Voting Records.csv', index=False)